In [1]:
## 设置数据库 & 定义数据模型
    
load './wtnb/wtnb.rb'
load './wtnb/wtmodel.rb'
load './wtnb/wtbms.rb'


IRuby.display IRuby.html(include_js)
IRuby.display IRuby.html(include_css)

ActiveRecord::Base.logger = nil

nil

connect_db("online")
enable_cache()

"<script src='https://unpkg.com/vue/dist/vue.js'></script>\n<script src='https://unpkg.com/chart.js@2.9.3/dist/Chart.bundle.js'></script>\n<script src='https://unpkg.com/vue-chartkick@0.5.1/dist/vue-chartkick.js'></script>\n<script src='https://unpkg.com/element-ui/lib/index.js'></script>\n"

"<link rel='stylesheet' href='https://unpkg.com/element-ui/lib/theme-chalk/index.css'>\n"

true

In [41]:
#获取预约商品列表

loop do 
    modify = false

    ret1 = BMS.api_call("https://console.wetry.shop/console/product/SelfSelection/getSelfSelectionProductList",{"currentPage": 1, "canMatchFlag": 1, "pageSize": 400})
    if ret1["data"]["list"]
        ret1["data"]["list"].each do |x|
            spu = x["spu"]
            size = x["size"]

            #通过商品获取订单
            ret2 = BMS.api_call("https://console.wetry.shop/console/order/order/getSaleList",{"currentPage": 1, "pageSize": 400, "spu": spu, "size": size})

            # 选type=3 更早的订单
            order_list = ret2["data"]["list"].map do |x| x["orderNo"] end
            order_no = Orders.where(order_no:order_list).where(type:3).order("type desc, create_time").where(status:5).first
            

            if order_no then

                # 添加到订单
                puts "添加到订单 #{order_no.order_no} #{spu}#{size}"
                BMS.addOrderItem(order_no.order_no,spu,size)
                modify = true
            end
        end
    end
    break unless modify

    puts "."
end